In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from datetime import datetime
from langchain_core.tools import tool


@tool
def get_current_time():
    """Use this to get current date or time"""
    return datetime.now()

In [ ]:
from typing import Annotated, TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

# 메모리 저장소 생성
memory = MemorySaver()

In [ ]:

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langgraph.prebuilt import create_react_agent

tools = [get_current_time]
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm = llm.bind_tools(tools)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer in Korean.",
        ),
        ("human", "{messages}"),
    ]
)


graph = create_react_agent(llm, tools, state_modifier=prompt, checkpointer=memory)

In [ ]:
from langchain_teddynote.graphs import visualize_graph

visualize_graph(graph, xray=True)

In [ ]:
from langchain_core.messages import HumanMessage

question = "한국의 수도는?"
inputs = {"messages": [("user", question)], }
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(
    recursion_limit=10,  # 최대 10개의 노드까지 방문. 그 이상은 RecursionError 발생
    configurable={"thread_id": "1"},  # 스레드 ID 설정
)

for chunk_msg, metadata  in graph.stream(inputs, stream_mode="messages", config=config):
    if (
            chunk_msg.content
            and not isinstance(chunk_msg, HumanMessage)
            and metadata["langgraph_node"] == "agent"
    ):
        print(chunk_msg.content, end="", flush=True)

In [ ]:
question = "그럼 일본은?"
inputs = {"messages": [("user", question)], }
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(
    recursion_limit=10,  # 최대 10개의 노드까지 방문. 그 이상은 RecursionError 발생
    configurable={"thread_id": "1"},  # 스레드 ID 설정
)

for chunk_msg, metadata  in graph.stream(inputs, stream_mode="messages", config=config):
    if (
            chunk_msg.content
            and not isinstance(chunk_msg, HumanMessage)
            and metadata["langgraph_node"] == "agent"
    ):
        print(chunk_msg.content, end="", flush=True)

In [ ]:
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(
    configurable={"thread_id": "1"},  # 스레드 ID 설정
)
# 그래프 상태 스냅샷 생성
snapshot = graph.get_state(config)
snapshot.values["messages"]

In [ ]:
from langchain_core.messages import HumanMessage

question = "현재 날짜는?"
inputs = {"messages": [("user", question)], }
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(
    recursion_limit=10,  # 최대 10개의 노드까지 방문. 그 이상은 RecursionError 발생
    configurable={"thread_id": "2"},  # 스레드 ID 설정
)

for chunk_msg, metadata  in graph.stream(inputs, stream_mode="messages", config=config):
    if (
            chunk_msg.content
            and not isinstance(chunk_msg, HumanMessage)
            and metadata["langgraph_node"] == "agent"
    ):
        print(chunk_msg.content, end="", flush=True)

In [ ]:
from langchain_core.messages import HumanMessage

question = "그럼 어제는?"
inputs = {"messages": [("user", question)], }
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(
    recursion_limit=10,  # 최대 10개의 노드까지 방문. 그 이상은 RecursionError 발생
    configurable={"thread_id": "2"},  # 스레드 ID 설정
)

for chunk_msg, metadata  in graph.stream(inputs, stream_mode="messages", config=config):
    if (
            chunk_msg.content
            and not isinstance(chunk_msg, HumanMessage)
            and metadata["langgraph_node"] == "agent"
    ):
        print(chunk_msg.content, end="", flush=True)

In [ ]:
config = RunnableConfig(
    configurable={"thread_id": "2"},  # 스레드 ID 설정
)
# 그래프 상태 스냅샷 생성
snapshot = graph.get_state(config)
snapshot.values["messages"]

In [ ]:
from langchain_core.messages import HumanMessage

question = "지금 몇일이야?"
inputs = {"messages": [("user", question)]}
for chunk_msg, metadata  in graph.stream(inputs, stream_mode="messages"):
    if (
            chunk_msg.content
            and not isinstance(chunk_msg, HumanMessage)
            and metadata["langgraph_node"] == "agent"
    ):
        print(chunk_msg.content, end="", flush=True)